In [1]:
import pandas as pd
import os
location = str(os.getcwd())

In [20]:
f = (location[:-12] + "Articles\\articles_cleaned.csv")
df = pd.read_csv(f)
df = df.to_dict("index")
print(len(df))

11768


In [5]:
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
stemmer = PorterStemmer()
np.random.seed(2018)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\wilmd\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for word in text:
        word.replace("\n", "")
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [7]:
df_clean = []
print(len(df))
for i in range(len(df)):
    body = df[i]["body"]
    if len(str(body)) > 100:
        if "rape" in body or "sexual" in body or "assault" in body or "misconduct" in body or "harassment" in body:
            df_clean.append(df[i])
print(len(df_clean))

11768
10884


In [8]:
df = pd.DataFrame(df_clean)
processed_docs = df['body'].map(preprocess)
processed_docs[:2]

0    [year, offic, women, affair, work, improv, liv...
1    [third, time, offic, student, associ, administ...
Name: body, dtype: object

In [9]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 accomplish
1 achiev
2 addit
3 address
4 affair
5 aim
6 alex
7 alyc
8 anniversari
9 assault
10 assensoh


In [10]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [11]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

In [12]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [15]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    from tqdm import tqdm

    for num_topics in tqdm(range(start, limit, step)):
        print(num_topics)
        model = gensim.models.LdaMulticore(corpus = corpus, num_topics = num_topics, id2word = dictionary, passes=2, workers=2)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=bow_corpus, texts=processed_docs, start=2, limit=15, step=1)

  0%|                                                                                                                                             | 0/13 [00:00<?, ?it/s]

2


  0%|                                                                                                                                             | 0/13 [00:03<?, ?it/s]


KeyboardInterrupt: 

In [14]:
coherence_values

[0.3200155663309372,
 0.3501589202633249,
 0.3710416963143046,
 0.3319701142107471,
 0.3886366105043175,
 0.34575733356444005,
 0.3791220873199293,
 0.36862315615513125,
 0.39069825247853124,
 0.39839163422759183,
 0.3833009113960846,
 0.3628397262538243,
 0.3895807344479638]

In [16]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=11, id2word=dictionary, passes=2, workers=2)

In [17]:
topic_df = []
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    topic_df.append({"index": idx, "topic_info": topic})
    
pd.DataFrame(topic_df).to_csv('topics_info.csv')

Topic: 0 
Words: 0.005*"women" + 0.005*"think" + 0.005*"right" + 0.004*"know" + 0.004*"stori" + 0.004*"film" + 0.003*"trump" + 0.003*"want" + 0.003*"life" + 0.003*"world"
Topic: 1 
Words: 0.005*"campu" + 0.004*"work" + 0.004*"citi" + 0.004*"want" + 0.004*"servic" + 0.004*"school" + 0.003*"night" + 0.003*"go" + 0.003*"state" + 0.003*"offic"
Topic: 2 
Words: 0.010*"campu" + 0.008*"work" + 0.008*"women" + 0.006*"issu" + 0.005*"commun" + 0.005*"group" + 0.005*"violenc" + 0.005*"organ" + 0.005*"presid" + 0.005*"member"
Topic: 3 
Words: 0.008*"campu" + 0.007*"commun" + 0.006*"school" + 0.005*"state" + 0.005*"work" + 0.004*"issu" + 0.004*"educ" + 0.004*"member" + 0.004*"need" + 0.004*"class"
Topic: 4 
Words: 0.005*"state" + 0.005*"season" + 0.005*"game" + 0.005*"team" + 0.005*"know" + 0.005*"play" + 0.004*"think" + 0.003*"work" + 0.003*"go" + 0.003*"love"
Topic: 5 
Words: 0.004*"music" + 0.004*"work" + 0.004*"think" + 0.004*"want" + 0.003*"know" + 0.003*"go" + 0.003*"album" + 0.003*"campu" + 

In [18]:
for index, score in sorted(lda_model[bow_corpus[7]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.3996129035949707	 
Topic: 0.004*"music" + 0.004*"work" + 0.004*"think" + 0.004*"want" + 0.003*"know" + 0.003*"go" + 0.003*"album" + 0.003*"campu" + 0.003*"thing" + 0.003*"take"

Score: 0.29256385564804077	 
Topic: 0.005*"campu" + 0.004*"work" + 0.004*"citi" + 0.004*"want" + 0.004*"servic" + 0.004*"school" + 0.003*"night" + 0.003*"go" + 0.003*"state" + 0.003*"offic"

Score: 0.23568807542324066	 
Topic: 0.006*"drink" + 0.005*"alcohol" + 0.005*"go" + 0.005*"women" + 0.004*"think" + 0.004*"help" + 0.004*"know" + 0.004*"need" + 0.004*"work" + 0.004*"drug"

Score: 0.04497963935136795	 
Topic: 0.005*"women" + 0.005*"think" + 0.005*"right" + 0.004*"know" + 0.004*"stori" + 0.004*"film" + 0.003*"trump" + 0.003*"want" + 0.003*"life" + 0.003*"world"

Score: 0.021929387003183365	 
Topic: 0.005*"state" + 0.005*"season" + 0.005*"game" + 0.005*"team" + 0.005*"know" + 0.005*"play" + 0.004*"think" + 0.003*"work" + 0.003*"go" + 0.003*"love"


Saving results to dataframe


In [30]:
df_clean = []

from tqdm import tqdm

for i in tqdm(range(len(df))):
    item = df[i]
    body = item["body"]
    url = item["url"]
    title = item["title"]
    location = item["location"]
    date = item["date"]
    processed_body = preprocess(str(body))
    bow_index = dictionary.doc2bow(processed_body)
    
    topic_list = []
    j = 0
    for index, score in sorted(lda_model[bow_index], key=lambda tup: -1*tup[1]):
        topic_list.append({"Score": score, "Topic": index, "Topic_terms": lda_model.print_topic(index, 10)})
        if j == 0:
            lead_topic = index
        j += 1
    
    if len(str(body)) > 100:
        if " rape " in body or " sexual " in body or " assault " in body or " misconduct " in body or " harassment " in body or " harass " in body:
            df_clean.append({"url": url, "body": body, "title": title, "location": location, "date": date, "topic scores": topic_list, "top_topic": lead_topic})
print(len(df_clean))

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11768/11768 [04:13<00:00, 46.36it/s]

8150


In [3]:
pd.DataFrame(df_clean).to_csv('articles_with_topic.csv')

NameError: name 'df_clean' is not defined

Deriving meaning from topics

In [4]:
f = (location + "\\articles_with_topic.csv")
df_clean = pd.read_csv(f)
df_clean = df_clean.to_dict("index")
print(len(df_clean))

8150


In [27]:
import ast

sorted_articles = []
for i in range(0,11):
    sorted_articles.append([])

for j in range(len(df_clean)):
    item = df_clean[j]
    topic_scores = ast.literal_eval(item["topic scores"])
    topic_id = topic_scores[0]["Topic"]
    sorted_articles[topic_id].append(item["url"])

In [28]:
import random
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
    print(len(sorted_articles[idx]))
    print(random.sample(sorted_articles[idx], 20))

NameError: name 'lda_model' is not defined

In [30]:
for i in range(0,11):
    print(i)
    print(len(sorted_articles[i]))
    print(random.sample(sorted_articles[i], 20))

0
1250
['https://stanforddaily.com/2019/11/19/the-foho-a-queer-startup-and-the-twisting-of-truth/', 'https://dbknews.com/0999/12/31/arc-ucfo4hjrpbapfe544pi27nj7rm/', 'https://stanforddaily.com/2015/04/22/indias-daughter-as-seen-by-americas-daughter/', 'https://stanforddaily.com/2017/10/30/the-vietnam-war-revisited-books-to-counteract-historical-narratives/', 'https://www.dailycardinal.com/article/2018/02/grammys-2018-recap-powerful-performances-are-marred-by-predictability', 'https://stanforddaily.com/2017/04/20/taking-delight-in-what-is-offensive/', 'https://www.dailycardinal.com/article/2013/04/minimalistic-speech-and-debate-wows', 'https://www.dailycardinal.com/article/2014/10/looking-at-celebrities-nudes-is-callous', 'https://dbknews.com/2016/08/12/article_a101d464-7109-11e4-bec8-d7a5e9493f7e-html/', 'https://www.idsnews.com/article/2016/04/editorial-clock-runs-out-for-kesha-in-new-york', 'https://dbknews.com/2014/12/07/article_7fa796a2-7e72-11e4-b727-9f4247b59907-html/', 'https://